In [1]:
import streamz

In [2]:
from gquant.dataframe_flow import Node
from gquant.dataframe_flow import NodePorts, PortsSpecSchema, TaskSpecSchema
from gquant.dataframe_flow import ConfSchema
from gquant.dataframe_flow import TaskGraph
from gquant.dataframe_flow.util import get_encoded_class

In [3]:
class StreamNode(Node):

    def ports_setup(self):
        input_ports = {}
        output_ports = {
            'stream_out': {
                PortsSpecSchema.port_type: streamz.Stream
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "Source node configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        columns_out = {
            'stream_out': {'element': 'number'
            },
        }
        return columns_out

    def process(self, inputs):
        output = {}
        output.update({'stream_out': streamz.Stream()})
        return output


In [4]:
class TransformNode(Node):

    def ports_setup(self):
        input_ports = {
             'stream_in': {
                PortsSpecSchema.port_type: streamz.Stream
            },           
        }
        output_ports = {
            'stream_out': {
                PortsSpecSchema.port_type: streamz.Stream
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "Transform Node configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'stream_in': {'element': 'number'}
        }
        columns_out = {
            'stream_out': {'element': 'number'
            },
        }
        return columns_out

    def process(self, inputs):
        in_stream = inputs['stream_in']
        output = {}
        def double(ele):
            return ele*2
        output.update({'stream_out': in_stream.map(double)})
        return output


In [5]:
class SinkNode(Node):

    def ports_setup(self):
        input_ports = {
             'stream_in': {
                PortsSpecSchema.port_type: streamz.Stream
            },           
        }
        output_ports = {
            'stream_out': {
                PortsSpecSchema.port_type: streamz.Stream
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "Simple SinkNode configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'stream_in': {'element': 'number'}
        }
        columns_out = {
            'stream_out': {'element': 'number'
            },
        }
        return columns_out

    def process(self, inputs):
        in_stream = inputs['stream_in']
        output = {}
        output.update({'stream_out': in_stream.sink(print)})
        return output

In [13]:
module_name = 'streamz'
source_spec = {
    TaskSpecSchema.task_id: 'source',
    TaskSpecSchema.node_type: StreamNode,
    TaskSpecSchema.conf: {},
    TaskSpecSchema.inputs: {},
    TaskSpecSchema.module: module_name
}


task_spec = {
    TaskSpecSchema.task_id: 'double',
    TaskSpecSchema.node_type: TransformNode,
    TaskSpecSchema.conf: {},
    TaskSpecSchema.inputs: {
        'stream_in': 'source.stream_out'
    },
    TaskSpecSchema.module: module_name
}

sink_spec = {
    TaskSpecSchema.task_id: 'sink',
    TaskSpecSchema.node_type: SinkNode,
    TaskSpecSchema.conf: {},
    TaskSpecSchema.inputs: {
        'stream_in': 'double.stream_out'
    },
    TaskSpecSchema.module: module_name
}

out_spec = {
    TaskSpecSchema.task_id: '',
    TaskSpecSchema.node_type: "Output_Collector",
    TaskSpecSchema.conf: {},
    TaskSpecSchema.inputs: {
        'in0': 'sink.stream_out',
        'in1': 'source.stream_out'
    }
}

task_list = [source_spec, task_spec, sink_spec, out_spec]

In [14]:
taskgraph = TaskGraph()
taskgraph.draw()

GQuantWidget(sub=HBox())

In [15]:
taskgraph.register_node(module_name, get_encoded_class(StreamNode))
taskgraph.register_node(module_name, get_encoded_class(TransformNode))
taskgraph.register_node(module_name, get_encoded_class(SinkNode))
task_graph = TaskGraph(task_list)
task_graph.draw()

GQuantWidget(sub=HBox(), value=[OrderedDict([('id', 'source'), ('type', 'StreamNode'), ('conf', {}), ('inputs'…

In [16]:
r = task_graph.run()

In [17]:
r.get_keys()

('sink.stream_out', 'source.stream_out')

In [18]:
r['source.stream_out'].emit(8)

16


In [19]:
import bqplot
import numpy as np
import bqplot.pyplot as plt

class PlotSinkNode(Node):

    def ports_setup(self):
        input_ports = {
             'stream_in': {
                PortsSpecSchema.port_type: streamz.Stream
            },           
        }
        output_ports = {
            'fig_out': {
                PortsSpecSchema.port_type: bqplot.Figure
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "Plot configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'stream_in': {'element': 'number'}
        }
        columns_out = {
            'fig_out': {}
        }
        return columns_out

    def process(self, inputs):
        in_stream = inputs['stream_in']
        axes_options = {'x': {'label': 'x'}, 'y': {'label': 'y'}}
        x = []
        y = []
        fig = plt.figure(animation_duration=10)
        lines = plt.plot(x=x, y=y, colors=['red', 'green'], axes_options=axes_options)
        output = {}
        def update(numbers):
            if isinstance(numbers[0], tuple):
                elements  = len(numbers[0])
            else:
                elements = len(numbers)
            x = np.arange(elements)
            y = np.array(numbers)
            lines.x = x
            lines.y = y
        in_stream.sink(update)
        output.update({'fig_out': fig})
        return output

In [20]:
class SlideWindowNode(Node):

    def ports_setup(self):
        input_ports = {
             'stream_in': {
                PortsSpecSchema.port_type: streamz.Stream
            },           
        }
        output_ports = {
            'stream_out': {
                PortsSpecSchema.port_type: streamz.Stream
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "SlideWindow Node configure",
            "type": "object",
            "properties": {
                "window": {
                    "type": "integer"
                }
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'stream_in': {'element': 'number'}
        }
        columns_out = {
            'stream_out': {'element': 'number'
            },
        }
        return columns_out

    def process(self, inputs):
        in_stream = inputs['stream_in']
        output = {}
        output.update({'stream_out': in_stream.sliding_window(self.conf['window'], return_partial=False)})
        return output

In [21]:
taskgraph.register_node(module_name, get_encoded_class(PlotSinkNode))
taskgraph.register_node(module_name, get_encoded_class(SlideWindowNode))

In [22]:
task_graph = TaskGraph.load_taskgraph('../taskgraphs/streamz/plot.gq.yaml')
task_graph.draw()

GQuantWidget(sub=HBox(), value=[OrderedDict([('id', 'source'), ('type', 'StreamNode'), ('conf', {}), ('inputs'…

In [23]:
r = task_graph.run()

In [24]:
r.get_keys()

('plot.fig_out', 'source.stream_out')

In [25]:
r['plot.fig_out']

Figure(animation_duration=10, axes=[Axis(label='x', scale=LinearScale(), side='bottom'), Axis(label='y', orien…

In [26]:
for i in range(1000):
    r['source.stream_out'].emit(np.sin(i/3.14))

In [27]:
class ZipNode(Node):

    def ports_setup(self):
        input_ports = {
             'stream1_in': {
                PortsSpecSchema.port_type: streamz.Stream
            }, 
             'stream2_in': {
                PortsSpecSchema.port_type: streamz.Stream
            },       
        }
        output_ports = {
            'stream_out': {
                PortsSpecSchema.port_type: streamz.Stream
            },
        }
        return NodePorts(inports=input_ports, outports=output_ports)

    def conf_schema(self):
        json = {
            "title": "Zip Node configure",
            "type": "object",
            "properties": {
            }
        }
        ui = {}
        return ConfSchema(json=json, ui=ui)

    def init(self):
        self.required = {}

    def columns_setup(self):
        self.required = {
            'stream1_in': {'element': 'number'},
            'stream2_in': {'element': 'number'}
        }
        columns_out = {
            'stream_out': {'element': 'number'
            },
        }
        return columns_out

    def process(self, inputs):
        in_stream1 = inputs['stream1_in']
        in_stream2 = inputs['stream2_in']
        output = {}
        output.update({'stream_out': in_stream1.zip(in_stream2)})
        return output

In [28]:
taskgraph.register_node(module_name, get_encoded_class(ZipNode))

In [29]:
task_graph = TaskGraph.load_taskgraph('../taskgraphs/streamz/two_branches.gq.yaml')
task_graph.draw()

GQuantWidget(sub=HBox(), value=[OrderedDict([('id', 'source'), ('type', 'StreamNode'), ('conf', {}), ('inputs'…

In [30]:
r = task_graph.run()

In [31]:
r.get_keys()

('source.stream_out', 'plot.fig_out')

In [32]:
r['plot.fig_out']

Figure(animation_duration=10, axes=[Axis(label='x', scale=LinearScale(), side='bottom'), Axis(label='y', orien…

In [33]:
for i in range(1000):
    r['source.stream_out'].emit(np.sin(i/3.14))